# Script to generate commands for Metagenomics for the AR Water Age Project

This code was written by Hannah Greenwald and Rose Kantor. Chunks in this code generate code for the command line to process reads, interact with Anvi'o, etc. Most of this work was conducted on Biotite, the Banfield lab server. This code is not comprehensive nor meant to be run linearly, and some of it generates output csv files to be run while other parts are directly written to be run at the command line to preserve what was run. Some parts of the code were meant for import into ggkbase and are not used for generation of data shown in the manuscript. See manuscript for more details on metagenomic analysis and the order of processing.

## Import and Directories

In [1]:
# conda install pandas
# conda update pandas

In [8]:
import pandas as pd
import numpy as np
import csv as csv
import os
import re
import glob as glob

In [9]:
idba_ud = '/shared/software/bin/idba_ud'
mash = '/shared/software/bin/mash'
bowtie2= '/shared/software/bin/bowtie2'
shrinksam= '/shared/software/bin/shrinksam'

assem_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies'
mash_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/mash_analysis'


In [10]:
assem_dict = {
    'ARSTAG_ARBF_1_post':["ARSTAG_AR_1_40", "ARSTAG_AR_1_41","ARSTAG_BF_1_40","ARSTAG_BF_1_41"],
    'ARSTAG_ARBF_2_post':["ARSTAG_AR_2_40", "ARSTAG_AR_2_41","ARSTAG_BF_2_40","ARSTAG_BF_2_41"],
    'ARSTAG_ARBF_3_post':["ARSTAG_AR_3_40", "ARSTAG_AR_3_41","ARSTAG_BF_3_40","ARSTAG_BF_3_41"],
    'ARSTAG_ARBF_4_post':["ARSTAG_AR_4_40", "ARSTAG_AR_4_41","ARSTAG_BF_4_40","ARSTAG_BF_4_41"],
    'ARSTAG_ARBF_5_post':["ARSTAG_AR_5_40", "ARSTAG_AR_5_41","ARSTAG_BF_5_40","ARSTAG_BF_5_41"],
    'ARSTAG_ARBF_12345_pre':["ARSTAG_AR_1_23", "ARSTAG_AR_1_27", "ARSTAG_AR_2_23", "ARSTAG_AR_2_27", "ARSTAG_AR_3_23", "ARSTAG_AR_3_27", "ARSTAG_AR_4_23", "ARSTAG_AR_5_23", "ARSTAG_AR_5_27"],
    'ARSTAG_AR_4_27':["ARSTAG_AR_4_27"],
    'ARSTAG_TAPRES_TAPRES_23':["ARSTAG_TAPRES_TAPRES_23"],
    'ARSTAG_TAPRES_TAPRES_27':["ARSTAG_TAPRES_TAPRES_27"],
    'ARSTAG_TAPRES_TAPRES_40':["ARSTAG_TAPRES_TAPRES_40"],
    'ARSTAG_TAPRES_TAPRES_41':["ARSTAG_TAPRES_TAPRES_41"],
    'ARSTAG_CONTROL_BFSLIDECONTROL_41':["ARSTAG_CONTROL_BFSLIDECONTROL_41"],
    'ARSTAG_CONTROL_MANIFB_41':["ARSTAG_CONTROL_MANIFB_41"],
    'ARSTAG_CONTROL_MOCK1E10_111821':["ARSTAG_CONTROL_MOCK1E10_111821"],
    'ARSTAG_CONTROL_MOCK1E8_111821':["ARSTAG_CONTROL_MOCK1E8_111821"]}



# Raw read processing

command for counting reads:
    
`seqkit stat *.fastq.gz > seqkit_output.tsv`

In [34]:
# combine the table you sent Rohan (with read names and project names) with the output from seqkit

In [35]:
df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})


In [7]:
#look at one entry to make sure it looks right
df.trim_file_F.values[0]

'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_TAPRES_TAPRES_23_trim_clean.PE.1.fastq.gz'

In [36]:
sample_id = [ARSTAG_TAPRES_TAPRES_23, ARSTAG_AR_1_23, ARSTAG_AR_2_23, ARSTAG_AR_3_23, ARSTAG_AR_4_23, ARSTAG_AR_5_23, 
             ARSTAG_TAPRES_TAPRES_27, ARSTAG_AR_1_27,ARSTAG_AR_2_27,ARSTAG_AR_3_27,ARSTAG_AR_4_27,ARSTAG_AR_5_27,
             ARSTAG_TAPRES_TAPRES_40, ARSTAG_AR_1_40, ARSTAG_AR_2_40, ARSTAG_AR_3_40, ARSTAG_AR_4_40, ARSTAG_AR_5_40,
             ARSTAG_BF_1_40, ARSTAG_BF_2_40, ARSTAG_BF_3_40, ARSTAG_BF_4_40, ARSTAG_BF_5_40,
             ARSTAG_TAPRES_TAPRES_41, ARSTAG_AR_1_41, ARSTAG_AR_2_41, ARSTAG_AR_3_41, ARSTAG_AR_4_41, ARSTAG_AR_5_41,
             ARSTAG_BF_1_41,ARSTAG_BF_2_41, ARSTAG_BF_3_41, ARSTAG_BF_4_41, ARSTAG_BF_5_41,
             ARSTAG_CONTROL_BFSLIDECONTROL_41, ARSTAG_CONTROL_MANIFB_41, ARSTAG_CONTROL_MOCK1E10_111821,ARSTAG_CONTROL_MOCK1E8_111821]

NameError: name 'ARSTAG_TAPRES_TAPRES_23' is not defined

# Assembly commands

In [37]:
# example of for loop where we iterate through the df row by row
df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})
for row in df.itertuples():
    s = row.sample_id
    print(s)


ARSTAG_TAPRES_TAPRES_23
ARSTAG_AR_1_23
ARSTAG_AR_2_23
ARSTAG_AR_3_23
ARSTAG_AR_4_23
ARSTAG_AR_5_23
ARSTAG_TAPRES_TAPRES_27
ARSTAG_AR_1_27
ARSTAG_AR_2_27
ARSTAG_AR_3_27
ARSTAG_AR_4_27
ARSTAG_AR_5_27
ARSTAG_TAPRES_TAPRES_40
ARSTAG_AR_1_40
ARSTAG_AR_2_40
ARSTAG_AR_3_40
ARSTAG_AR_4_40
ARSTAG_AR_5_40
ARSTAG_TAPRES_TAPRES_41
ARSTAG_AR_1_41
ARSTAG_AR_2_41
ARSTAG_AR_3_41
ARSTAG_AR_4_41
ARSTAG_AR_5_41
ARSTAG_BF_1_40
ARSTAG_BF_2_40
ARSTAG_BF_3_40
ARSTAG_BF_4_40
ARSTAG_BF_5_40
ARSTAG_BF_1_41
ARSTAG_BF_2_41
ARSTAG_BF_3_41
ARSTAG_BF_4_41
ARSTAG_BF_5_41
ARSTAG_CONTROL_BFSLIDECONTROL_41
ARSTAG_CONTROL_MANIFB_41
ARSTAG_CONTROL_MOCK1E10_111821
ARSTAG_CONTROL_MOCK1E8_111821
ARSTAG_AR_5_23
ARSTAG_AR_5_27
ARSTAG_AR_4_40
ARSTAG_BF_1_41
ARSTAG_TAPRES_TAPRES_41


In [35]:
# we will create an empty list, iterate thru the df and generate commands, save them in the list, then turn the list into a column in our df
idba_cmd = []
for row in df.itertuples():
    cmd = f'sbatch --wrap "' \
          f'{idba_ud} ' \
          f'-r {row.combined_file} ' \
          f'-o {assem_dir}/{row.sample_id}.idba_ud"'

    idba_cmd.append(cmd)
idba_cmd = pd.Series(idba_cmd, name='idba_cmd')
df['idba_cmd'] = idba_cmd

In [37]:
# we can check the first line to see that the command worked
idba_cmd[0]

'sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/sequences/2022/ARSTAG_TAPRES_TAPRES_23/raw.d/ARSTAG_TAPRES_TAPRES_23_trim_clean.PE.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud"'

In [38]:
# write the commands to a CSV
df['idba_cmd'].to_csv('/Users/rosekantor/data/metagenomics_mentoring/ARSTAG/assembly.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

note: alternatively you could just open a file and write the commands directly to the file as you create them (in the for loop)

Now push this shell script up to biotite and run it: `sh assembly.sh`

### Coassembly of tap reservoir samples

In [ ]:
#concatenate fa files together and put into new folder "combined"

cat /groups/banfield/sequences/2022/ARSTAG_TAPRES_TAPRES*/raw.d/ARSTAG_TAPRES_TAPRES*_trim_clean.PE.fa > tapres_coassembly_reads.fa



In [ ]:
#idba assembly 
sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/tapres_coassembly_reads.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/tapres_coassembly.idba_ud"


In [ ]:
#navigate to proper directory
cd f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/tapres_coassembly.idba_ud'
#remove unnecessary files
rm kmer contig-* align-* graph-* local-*
#change headers
sed 's/scaffold/tapres_coassembly/' scaffold.fa > tapres_coassembly_scaffold.fa
#get stats on assembly
contig_stats.pl -i tapres_coassembly_scaffold.fa

#then download quality file locally into right folder
# scp 'hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/tapres_coassembly.idba_ud/tapres_coassembly_scaffold.fa.summary.txt' ./


### Clean up

In [27]:
clean_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    #remove unnecessary files
    del_cmd = f'rm {assem_dir}/{row.sample_id}.idba_ud/kmer ' \
              f'{assem_dir}/{row.sample_id}.idba_ud/contig-* ' \
              f'{assem_dir}/{row.sample_id}.idba_ud/align-* ' \
              f'{assem_dir}/{row.sample_id}.idba_ud/graph-* ' \
              f'{assem_dir}/{row.sample_id}.idba_ud/local-*' 
    
    #change headers
    rename_cmd = f"sed 's/scaffold/{row.sample_id}/' {assem_dir}/{row.sample_id}.idba_ud/scaffold.fa > {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold.fa" 
    
    #get stats on assembly
    stats_cmd = f'contig_stats.pl -i {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold.fa'
    
    #don't do this in the future
    move_stats_cmd =  f'mv {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold.fa.summary.txt {assem_dir}/idba_contig_stats_summary/{row.sample_id}_scaffold.fa.summary.txt'
    
    cmd = [del_cmd, rename_cmd, stats_cmd, move_stats_cmd]
    all_cmd = '; '.join(cmd) # separate all commands by semicolon (so they will be executed in order for each sample)
    clean_cmd.append(all_cmd) # append command to list
        

clean_cmd = pd.Series(clean_cmd, name='clean_cmd')
df['clean_cmd'] = clean_cmd



In [28]:
#write commands to CSV
df['clean_cmd'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/clean.sh', index=False, quoting=csv.QUOTE_NONE, header=False)


In [29]:
df['clean_cmd'][4]

#test run with just this one sample by copy and pasting below
#then upload csv file to server
#then run script by typing "sh clean.sh"

"rm /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/kmer /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/contig-* /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/align-* /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/graph-* /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/local-*; sed 's/scaffold/ARSTAG_AR_4_23/' /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/scaffold.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/ARSTAG_AR_4_23_scaffold.fa; contig_stats.pl -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_23.idba_ud/ARSTAG_AR_4_23_scaffold.fa; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/

In [65]:
#should have done this in for loop above but now have to do it here (rose ended up doing all of this for individual assemblies)

postassem_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    # filter for only contigs ≥1000 bp
    min1000 = f'pullseq -i {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold.fa --min 1000 > {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold_min1000.fa'
    
    # make directory to store bowtie2 indices in
    mdbt2 = f'mkdir {assem_dir}/{row.sample_id}.idba_ud/bt2/'

    # index in prep for bowtie2 mapping
    ind = f'bowtie2-build {assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold_min1000.fa {assem_dir}/{row.sample_id}.idba_ud/bt2/{row.sample_id}_scaffold_min1000.fa'
   
    cmd = [min1000, mdbt2, ind]
    all_cmd = '; '.join(cmd) # separate all commands by semicolon (so they will be executed in order for each sample)
    postassem_cmd.append(all_cmd) # append command to list
        

postassem_cmd = pd.Series(postassem_cmd, name='postassem_cmd')
df['postassem_cmd'] = postassem_cmd



AttributeError: 'Pandas' object has no attribute 'sample_id'

In [ ]:
#write commands to CSV
df['postassem_cmd'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/postassem.sh', index=False, quoting=csv.QUOTE_NONE, header=False)


## Reactor Coassembly

In [ ]:
#concatenate fa files together and put into new folder "combined"

#group 1: AR1_40, AR1_41, BF1_40, BF1_41 --> ARSTAG_ARBF_1_post
cat /groups/banfield/sequences/2022/ARSTAG_AR_1_40/raw.d/ARSTAG_AR_1_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_1_41/raw.d/ARSTAG_AR_1_41_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_1_40/raw.d/ARSTAG_BF_1_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_1_41/raw.d/ARSTAG_BF_1_41_trim_clean.PE.fa > ARSTAG_ARBF_1_post.fa

#group 2:AR2_40, AR2_41, BF2_40, BF2_41 --> ARSTAG_ARBF_2_post
cat /groups/banfield/sequences/2022/ARSTAG_AR_2_40/raw.d/ARSTAG_AR_2_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_2_41/raw.d/ARSTAG_AR_2_41_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_2_40/raw.d/ARSTAG_BF_2_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_2_41/raw.d/ARSTAG_BF_2_41_trim_clean.PE.fa > ARSTAG_ARBF_2_post.fa

#group 3: AR3_40, AR3_41, BF3_40, BF3_41 --> ARSTAG_ARBF_3_post
cat /groups/banfield/sequences/2022/ARSTAG_AR_3_40/raw.d/ARSTAG_AR_3_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_3_41/raw.d/ARSTAG_AR_3_41_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_3_40/raw.d/ARSTAG_BF_3_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_3_41/raw.d/ARSTAG_BF_3_41_trim_clean.PE.fa > ARSTAG_ARBF_3_post.fa

#group 4:AR4_40, AR4_41, BF4_40, BF4_41 --> ARSTAG_ARBF_4_post
cat /groups/banfield/sequences/2022/ARSTAG_AR_4_40/raw.d/ARSTAG_AR_4_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_4_41/raw.d/ARSTAG_AR_4_41_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_4_40/raw.d/ARSTAG_BF_4_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_4_41/raw.d/ARSTAG_BF_4_41_trim_clean.PE.fa > ARSTAG_ARBF_4_post.fa

#group 5:AR5_40, AR5_41, BF5_40, BF5_41 --> ARSTAG_ARBF_5_post
cat /groups/banfield/sequences/2022/ARSTAG_AR_5_40/raw.d/ARSTAG_AR_5_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_5_41/raw.d/ARSTAG_AR_5_41_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_5_40/raw.d/ARSTAG_BF_5_40_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_BF_5_41/raw.d/ARSTAG_BF_5_41_trim_clean.PE.fa > ARSTAG_ARBF_5_post.fa

#group 6: all ARs from time points 23 and 27 except ARSTAG_AR_4_27 --> ARSTAG_AR_12345_pre
cat /groups/banfield/sequences/2022/ARSTAG_AR_1_23/raw.d/ARSTAG_AR_1_23_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_1_27/raw.d/ARSTAG_AR_1_27_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_2_23/raw.d/ARSTAG_AR_2_23_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_2_27/raw.d/ARSTAG_AR_2_27_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_3_23/raw.d/ARSTAG_AR_3_23_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_3_27/raw.d/ARSTAG_AR_3_27_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_4_23/raw.d/ARSTAG_AR_4_23_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_5_23/raw.d/ARSTAG_AR_5_23_trim_clean.PE.fa /groups/banfield/sequences/2022/ARSTAG_AR_5_27/raw.d/ARSTAG_AR_5_27_trim_clean.PE.fa > ARSTAG_ARBF_12345_pre.fa 

#individual assemblies being used: each tapres sample, each control, ARSTAG_AR_4_27




In [ ]:
#manually creating commands for assembly

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_1_post.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud"

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_2_post.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud"

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_3_post.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud"

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_4_post.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud"

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_5_post.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud"

sbatch --wrap "/shared/software/bin/idba_ud -r /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/combined/ARSTAG_ARBF_12345_pre.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud"



In [142]:
#post assembly

coass_names = ["ARSTAG_ARBF_1_post","ARSTAG_ARBF_2_post","ARSTAG_ARBF_3_post","ARSTAG_ARBF_4_post","ARSTAG_ARBF_5_post", "ARSTAG_ARBF_12345_pre" ]
postassem_cmd = []

for s in coass_names:
    
    #remove unnecessary files
    del_cmd = f'rm {assem_dir}/{s}.idba_ud/kmer ' \
              f'{assem_dir}/{s}.idba_ud/contig-* ' \
              f'{assem_dir}/{s}.idba_ud/align-* ' \
              f'{assem_dir}/{s}.idba_ud/graph-* ' \
              f'{assem_dir}/{s}.idba_ud/local-*' 
    
    #change headers
    rename_cmd = f"sed 's/scaffold/{s}/' {assem_dir}/{s}.idba_ud/scaffold.fa > {assem_dir}/{s}.idba_ud/{s}_scaffold.fa" 
    
    #get stats on assembly
    stats_cmd = f'contig_stats.pl -i {assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    
    # make directory to store bowtie2 indices in
    mdbt2 = f'mkdir {assem_dir}/{s}.idba_ud/bt2/'

    # index in prep for bowtie2 mapping
    ind = f'bowtie2-build {assem_dir}/{s}.idba_ud/{s}_scaffold.fa {assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa'
   
    
    cmd = [del_cmd, rename_cmd, stats_cmd, mdbt2, ind]
    all_cmd = '; '.join(cmd) # separate all commands by semicolon (so they will be executed in order for each sample)
    postassem_cmd.append(all_cmd) # append command to list
        

postassem_cmd = pd.Series(postassem_cmd, name='postassem_cmd')




In [173]:
#write commands to CSV
postassem_cmd.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/3c.postassem.sh', index=False, quoting=csv.QUOTE_NONE, header=False)


In [181]:
# #create variables for raw read paths for each coassembly group

# coass1_readF = ''
# coass1_readR = ''
# for s in ["ARSTAG_AR_1_40", "ARSTAG_AR_1_41","ARSTAG_BF_1_40","ARSTAG_BF_1_41"]:
#     if len(coass1_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass1_readF=coass1_readF+read_pathF
#     coass1_readR=coass1_readR+read_pathR
    
# coass2_readF = ''
# coass2_readR = ''
# for s in ["ARSTAG_AR_2_40", "ARSTAG_AR_2_41","ARSTAG_BF_2_40","ARSTAG_BF_2_41"]:
#     if len(coass2_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass2_readF=coass2_readF+read_pathF
#     coass2_readR=coass2_readR+read_pathR

# coass3_readF = ''
# coass3_readR = ''
# for s in ["ARSTAG_AR_3_40", "ARSTAG_AR_3_41","ARSTAG_BF_3_40","ARSTAG_BF_3_41"]:
#     if len(coass3_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass3_readF=coass3_readF+read_pathF
#     coass3_readR=coass3_readR+read_pathR
    
# coass4_readF = ''
# coass4_readR = ''
# for s in ["ARSTAG_AR_4_40", "ARSTAG_AR_4_41","ARSTAG_BF_4_40","ARSTAG_BF_4_41"]:
#     if len(coass4_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass4_readF=coass4_readF+read_pathF
#     coass4_readR=coass4_readR+read_pathR
    
# coass5_readF = ''
# coass5_readR = ''
# for s in ["ARSTAG_AR_5_40", "ARSTAG_AR_5_41","ARSTAG_BF_5_40","ARSTAG_BF_5_41"]:
#     if len(coass5_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass5_readF=coass5_readF+read_pathF
#     coass5_readR=coass5_readR+read_pathR
    

# coass6_readF = ''
# coass6_readR = ''
# for s in ["ARSTAG_AR_1_23", "ARSTAG_AR_1_27", "ARSTAG_AR_2_23", "ARSTAG_AR_2_27", "ARSTAG_AR_3_23", "ARSTAG_AR_3_27", "ARSTAG_AR_4_23", "ARSTAG_AR_5_23", "ARSTAG_AR_5_27"]:
#     if len(coass6_readF) == 0: 
#         read_pathF = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     else:
#         read_pathF = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.1.fastq.gz'
#         read_pathR = f',/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/{s}_trim_clean.PE.2.fastq.gz'
#     coass6_readF=coass6_readF+read_pathF
#     coass6_readR=coass6_readR+read_pathR


    



In [182]:
coass6_readR

'/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_1_23_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_1_27_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_2_23_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_2_27_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_3_23_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_3_27_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_23_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_5_23_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARST

In [183]:
# # mapping commands for coassembly (Rose did it on individual assemblies)

# data = {'coass_names':["ARSTAG_ARBF_1_post","ARSTAG_ARBF_2_post","ARSTAG_ARBF_3_post","ARSTAG_ARBF_4_post","ARSTAG_ARBF_5_post", "ARSTAG_ARBF_12345_pre" ],
#         'Freads': [coass1_readF,coass2_readF,coass3_readF,coass4_readF,coass5_readF,coass6_readF],
#         'Rreads':[coass1_readR,coass2_readR,coass3_readR,coass4_readR,coass5_readR,coass6_readR]}
# df= pd.DataFrame(data)

# mapping_cmd = []

# for row in df.itertuples():
#     s = row.coass_names
#     F = row.Freads
#     R = row.Rreads

#     cmd = f'sbatch --wrap "' \
#           f'{bowtie2} ' \
#           f'-p 48 -x {assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa ' \
#           f'-1 {F} -2 {R} ' \
#           f'2> {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.log ' \
#           f'| {shrinksam} -v > {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam"'

#     mapping_cmd.append(cmd)

# mapping_cmd = pd.Series(mapping_cmd, name='mapping_cmd')


In [184]:
mapping_cmd[3]

'sbatch --wrap "/shared/software/bin/bowtie2 -p 48 -x /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/bt2/ARSTAG_ARBF_4_post_scaffold.fa -1 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_40_trim_clean.PE.1.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_41_trim_clean.PE.1.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_BF_4_40_trim_clean.PE.1.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_BF_4_41_trim_clean.PE.1.fastq.gz -2 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_40_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_41_trim_clean.PE.2.fastq.gz,/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_BF_4_40_trim_clean.PE.2.fastq.gz,/

In [185]:
# #write commands to CSV
# mapping_cmd.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/4c.mapping.sh', index=False, quoting=csv.QUOTE_NONE, header=False, escapechar='\\')


In [200]:
bt2 = '/shared/software/bin/bowtie2'
reads_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed'
coass_dict = {
    'ARSTAG_ARBF_1_post':["ARSTAG_AR_1_40", "ARSTAG_AR_1_41","ARSTAG_BF_1_40","ARSTAG_BF_1_41"],
    'ARSTAG_ARBF_2_post':["ARSTAG_AR_2_40", "ARSTAG_AR_2_41","ARSTAG_BF_2_40","ARSTAG_BF_2_41"],
    'ARSTAG_ARBF_3_post':["ARSTAG_AR_3_40", "ARSTAG_AR_3_41","ARSTAG_BF_3_40","ARSTAG_BF_3_41"],
    'ARSTAG_ARBF_4_post':["ARSTAG_AR_4_40", "ARSTAG_AR_4_41","ARSTAG_BF_4_40","ARSTAG_BF_4_41"],
    'ARSTAG_ARBF_5_post':["ARSTAG_AR_5_40", "ARSTAG_AR_5_41","ARSTAG_BF_5_40","ARSTAG_BF_5_41"],
    'ARSTAG_ARBF_12345_pre':["ARSTAG_AR_1_23", "ARSTAG_AR_1_27", "ARSTAG_AR_2_23", "ARSTAG_AR_2_27", "ARSTAG_AR_3_23", "ARSTAG_AR_3_27", "ARSTAG_AR_4_23", "ARSTAG_AR_5_23", "ARSTAG_AR_5_27"]}


In [201]:
#mapping for ggkbase for coassembly 

mapping_cmd = []
for k,v in coass_dict.items():
    s = k
    f_reads = []
    r_reads = []
    for r in v:
        f_reads.append(f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz')
        r_reads.append(f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz')
    f_reads = ','.join(f_reads)
    r_reads = ','.join(r_reads)

    cmd = f'sbatch --wrap "' \
          f'{bt2} ' \
          f'-p 48 -x {assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa ' \
          f'-1 {f_reads} -2 {r_reads} ' \
          f'2> {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.log ' \
          f'| {shrinksam} -v > {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam"'

    mapping_cmd.append(cmd)

df['mapping_cmd'] = mapping_cmd
df['mapping_cmd'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/4c.mapping.sh', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)

In [204]:
# readcounts commands for ggkbase for coassembly
readcounts = '/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb'

readcounts_cmd = []
for k,v in coass_dict.items():
    s = k
    
    #count reads and insert in header?
    add_readcount= f'{readcounts} {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam {assem_dir}/{s}.idba_ud/{s}_scaffold.fa 150 > {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted'
    
    #rename the file with counts in the header to just scaffold (gets rid of the other one)
    cmd_mv = f'mv {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted {assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    
    # filter for only contigs ≥1000 bp
    min1000 = f'pullseq -i {assem_dir}/{s}.idba_ud/{s}_scaffold.fa --min 1000 > {assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'
    
    readcounts_cmd.append(f'{add_readcount}; {cmd_mv}; {min1000}')
    
readcounts_cmd = pd.Series(readcounts_cmd, name='readcounts_cmd')
readcounts_cmd.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/5c.readcounts.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

  

In [206]:
# gene calling for ggkbase for coassembly

genecalls_cmd = []
for k,v in coass_dict.items():
    s = k
    
    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    scaffolds_min1000 = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'
    
    call_orfs = f'prodigal -i {scaffolds_min1000} -o {scaffolds_min1000}.genes -a {scaffolds_min1000}.genes.faa -d {scaffolds_min1000}.genes.fna -m -p meta'
    call_16S = f'/groups/banfield/software/pipeline/v1.1/scripts/16s.sh {scaffolds_min1000} > {scaffolds_min1000}.16s'
    call_trnas = f'/groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i {scaffolds_min1000} > /dev/null 2>&1'
    
    genecalls_cmd.append(f'{call_orfs}; {call_16S}; {call_trnas}')

genecalls_cmd = pd.Series(genecalls_cmd, name='genecalls_cmd')
genecalls_cmd.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/6c.genecalls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)


### update quality table after assembly

In [7]:
#import table and add rows for coassemblies 

df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})


In [8]:
pat_count = r'Total number of sequences: (\d+)'
pat_n50 = r'N50:\s+(\d+)'
pat_size = r'Total number of bps:\s+(\d+)'

In [9]:
# find post-assembly quality parameters within quality files and add to quality table

#download quality files locally
ass_quality_path = "/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/idba_contig_stats_summary/"
os.chdir(ass_quality_path)

df["N50"]= ""
df["scaffold_count"]= ""
df["assembly_size"]= ""

for row in df.itertuples():
    sample_id = row.sample_id
    r= row.Index
    with open(f'{sample_id}_scaffold.fa.summary.txt', 'r') as f:
#         for line in f.readlines():
#             if 'N50' in line:
#                 read = line.split(' ')
#                 N50_value= read[23]
#                 df.loc['r', 'N50'] = N50
                
        for line in f.readlines():
            if 'N50' in line:
                N50= re.match(pat_n50, line).group(1)
#                 df["N50"][r] = N50
                df.loc[r, 'N50'] = N50
        
            if 'Total number of sequences' in line:
                scaffold_count= re.match(pat_count, line).group(1)
#                 df["scaffold_count"][r] = scaffold_count
                df.loc[r,'scaffold_count'] = scaffold_count
        
            if 'Total number of bps' in line:
                assembly_size= re.match(pat_size, line).group(1)
#                 df["scaffold_size"][r] = scaffold_size
                df.loc[r,'assembly_size'] = assembly_size
        
#             if 'Total number of sequences' in line:
#                 read = line.split(' ')
#                 postass_seqs= read[4]
#                 size= len(postass_seqs)
#                 postass_seqs2 = postass_seqs[:size - 2]
#                 df["postass_seqs"][r]= postass_seqs2
                
    
df.loc[:,"N50"]= pd.to_numeric(df["N50"])
df.loc[:,"scaffold_count"]= pd.to_numeric(df["scaffold_count"])
df.loc[:,"assembly_size"]= pd.to_numeric(df["assembly_size"])


In [10]:
df

,sample_id,raw_file_F,raw_file_R,trim_file_F,trim_file_R,raw_num_seq_F,raw_num_seq_R,trim_num_seq_F,trim_num_seq_R,perc_seq_lost_to_trim,trim_bpsF,trim_bpsR,trim_bps,combined_file,N50,scaffold_count,assembly_size
0,ARSTAG_TAPRES_TAPRES_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,38735372,38735372,38616183,38616183,0.307701,5714577182,5717272608,11431849790,/groups/banfield/sequences/2022/ARSTAG_TAPRES_...,1322,554239,511557867
1,ARSTAG_AR_1_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,26391193,26391193,26273730,26273730,0.445084,3862453098,3863143875,7725596973,/groups/banfield/sequences/2022/ARSTAG_AR_1_23...,16075,76242,148886568
2,ARSTAG_AR_2_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,37543974,37543974,37390867,37390867,0.407807,5488794620,5486098408,10974893028,/groups/banfield/sequences/2022/ARSTAG_AR_2_23...,18682,79234,179057845
3,ARSTAG_AR_3_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,42645590,42645590,42449515,42449515,0.459778,6261247154,6263941418,12525188572,/groups/banfield/sequences/2022/ARSTAG_AR_3_23...,18632,98036,184327829
4,ARSTAG_AR_4_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,29074701,29074701,28936051,28936051,0.476875,4142238876,4142289753,8284528629,/groups/banfield/sequences/2022/ARSTAG_AR_4_23...,6744,95368,157358641
5,ARSTAG_AR_5_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,28011485,28011485,27871385,27871385,0.500152,4046405618,4048514411,8094920029,/groups/banfield/sequences/2022/ARSTAG_AR_5_23...,6748,108426,185939560
6,ARSTAG_TAPRES_TAPRES_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,33761415,33761415,33603683,33603683,0.467196,4906022542,4911716747,9817739289,/groups/banfield/sequences/2022/ARSTAG_TAPRES_...,878,284917,216044543
7,ARSTAG_AR_1_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,27493842,27493842,27377695,27377695,0.422447,4039802210,4041261360,8081063570,/groups/banfield/sequences/2022/ARSTAG_AR_1_27...,10576,78893,153614428
8,ARSTAG_AR_2_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,34751616,34751616,34581359,34581359,0.489925,5123709949,5113616600,10237326549,/groups/banfield/sequences/2022/ARSTAG_AR_2_27...,10983,102348,186453734
9,ARSTAG_AR_3_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,50243555,50243555,49984555,49984555,0.515489,7379900348,7375406833,14755307181,/groups/banfield/sequences/2022/ARSTAG_AR_3_27...,19962,113300,195143840


In [66]:
#write commands to CSV
df.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/sample_quality_table_postass.csv')


ImportError: cannot import name 'CompressionOptions' from 'pandas._typing' (/Users/hannahgreenwald/anaconda3/lib/python3.8/site-packages/pandas/_typing.py)

In [12]:
#for coassemblies 
ass_quality_path = "/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/idba_contig_stats_summary/"
os.chdir(ass_quality_path)

assembly_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']

df= pd.DataFrame()
df["sample_id"] = assembly_id

df["N50"]= ""
df["scaffold_count"]= ""
df["assembly_size"]= ""

for row in df.itertuples():
    sample_id = row.sample_id
    r= row.Index
    with open(f'{sample_id}_scaffold.fa.summary.txt', 'r') as f:
#         for line in f.readlines():
#             if 'N50' in line:
#                 read = line.split(' ')
#                 N50_value= read[23]
#                 df.loc['r', 'N50'] = N50
                
        for line in f.readlines():
            if 'N50' in line:
                N50= re.match(pat_n50, line).group(1)
#                 df["N50"][r] = N50
                df.loc[r, 'N50'] = N50
        
            if 'Total number of sequences' in line:
                scaffold_count= re.match(pat_count, line).group(1)
#                 df["scaffold_count"][r] = scaffold_count
                df.loc[r,'scaffold_count'] = scaffold_count
        
            if 'Total number of bps' in line:
                assembly_size= re.match(pat_size, line).group(1)
#                 df["scaffold_size"][r] = scaffold_size
                df.loc[r,'assembly_size'] = assembly_size
        
#             if 'Total number of sequences' in line:
#                 read = line.split(' ')
#                 postass_seqs= read[4]
#                 size= len(postass_seqs)
#                 postass_seqs2 = postass_seqs[:size - 2]
#                 df["postass_seqs"][r]= postass_seqs2
                
    
df.loc[:,"N50"]= pd.to_numeric(df["N50"])
df.loc[:,"scaffold_count"]= pd.to_numeric(df["scaffold_count"])
df.loc[:,"assembly_size"]= pd.to_numeric(df["assembly_size"])

In [14]:
df.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/assem_quality_table_postass.csv')



In [50]:
#create column with paths to assemblies for ggkbase table import
# pathtoass = ''
# delim= ','

# for row in df.itertuples():
#     path = f'{assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold_min1000.fa'
#     pathtoass += (path + delim)

pathtoass = []
for row in df.itertuples():
    path = f'{assem_dir}/{row.sample_id}.idba_ud/{row.sample_id}_scaffold_min1000.fa'
    pathtoass.append(path)
    
pathtoass
pathtoass = pd.Series(pathtoass, name='pathtoass')
pathtoass.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/pathtoass.csv', index=False, quoting=csv.QUOTE_NONE, header=False)


In [ ]:
#tmux! done
#step 5 for coassemblies
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa

In [ ]:
#step 6 coassemblies
#tmux!! done
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa > /dev/null 2>&1
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa > /dev/null 2>&1
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa > /dev/null 2>&1
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa > /dev/null 2>&1
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa > /dev/null 2>&1
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa > /dev/null 2>&1


In [ ]:
# for coassemblies after mapping (remove .sam files that take up a ton of space)

rm BASE_NAME_scaffold_mapped.sam



In [ ]:
#step 7 everything

sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"


In [ ]:
#tmux 
#step 8 everything, submitted tmux

gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/ARSTAG_ARBF_1_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/ARSTAG_ARBF_2_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_3_post.idba_ud/ARSTAG_ARBF_3_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_4_post.idba_ud/ARSTAG_ARBF_4_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_5_post.idba_ud/ARSTAG_ARBF_5_post_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_23.idba_ud/ARSTAG_TAPRES_TAPRES_23_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_27.idba_ud/ARSTAG_TAPRES_TAPRES_27_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_40.idba_ud/ARSTAG_TAPRES_TAPRES_40_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_TAPRES_TAPRES_41.idba_ud/ARSTAG_TAPRES_TAPRES_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_BFSLIDECONTROL_41.idba_ud/ARSTAG_CONTROL_BFSLIDECONTROL_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MANIFB_41.idba_ud/ARSTAG_CONTROL_MANIFB_41_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+
gzip /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa*.b6; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-kegg.b6.gz kegg > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-kegg.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-uni.b6.gz uniref > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-uni.b6+; /shared/software/bin/annolookup.py /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-uniprot.b6.gz uniprot > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E8_111821.idba_ud/ARSTAG_CONTROL_MOCK1E8_111821_scaffold_min1000.fa.genes.faa-vs-uniprot.b6+


## Fixing AR_4_27

In [ ]:
#fixing AR_4_27 (for some reason all files after scaffold.fa are blank)

sed 's/scaffold/ARSTAG_AR_4_27/' scaffold.fa > ARSTAG_AR_4_27_scaffold.fa #run, not empty now
contig_stats.pl -i ARSTAG_AR_4_27_scaffold.fa #run

#self-mapping  #run
sbatch --wrap "/shared/software/bin/bowtie2 -p 48 -x /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/bt2/ARSTAG_AR_4_27_scaffold.fa -1 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_27_trim_clean.PE.1.fastq.gz -2 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_4_27_trim_clean.PE.2.fastq.gz 2> /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_mapped.log | /shared/software/bin/shrinksam -v > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_mapped.sam"

#readcounts, run
/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_mapped.sam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold.fa 150 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold.fa.counted; mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold.fa.counted /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold.fa; pullseq -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold.fa --min 1000 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa

#annotation, run
prodigal -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes -a /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.fna -m -p meta; /groups/banfield/software/pipeline/v1.1/scripts/16s.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.16s; /groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa > /dev/null 2>&1

#usearch, running
sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d kegg --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d uni --nocluster"; sbatch --wrap "/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa.genes.faa -k -d uniprot --nocluster"

#anvio steps, DON'T FORGET TMUX, done
awk '{print $1}' /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_scaffold_min1000.fa > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_min1000.fa

#then rerun anvio command 1b and 2 for everything, submitted 1b
#then run step 8 for everything (code below), submitted 
#then crossmapping


# Mash commands

In [41]:
# mash all v. all reads
mash_cmd = []
for row in df.itertuples():
    cmd = f'cat {row.trim_file_F} {row.trim_file_R} | {mash} sketch -m 2 -r - -I {row.sample_id} -s 10000 -o {mash_dir}/{row.sample_id}'
    mash_cmd.append(cmd)
    
mash_cmd = pd.Series(mash_cmd, name='mash_cmd')
df['mash_cmd'] = mash_cmd

In [43]:
# write the commands to a CSV
df['mash_cmd'].to_csv('/Users/rosekantor/data/metagenomics_mentoring/ARSTAG/mash.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

## mash dist commands
`mash paste` combines multiple sketches into a single sketch.  The first arg is output name, followed by a list of all the sketch files you want to combine

Command: `mash paste arstag.msh *msh`

`mash dist` can sketch on the fly or take a sketch as input.  Because we are doing all-vs-all we use the same msh file as the query and reference.

Command: `mash dist arstag.msh arstag.msh > arstag_mashdist.csv`

In [ ]:
## mash dist commands excluding mock communities
`mash paste` combines multiple sketches into a single sketch.  The first arg is output name, followed by a list of all the sketch files you want to combine

#copy the folder and then remove the mock files from it
Command: `mash paste arstag.msh *msh`

`mash dist` can sketch on the fly or take a sketch as input.  Because we are doing all-vs-all we use the same msh file as the query and reference.

Command: `mash dist arstag.msh arstag.msh > arstag_mashdist_nomock.csv`

# Anvio

In [ ]:
# need to take away headers and make new file name

# pullseq -i ARSTAG_CONTROL_MOCK1E10_111821_scaffold.fa --min 1000 > ARSTAG_CONTROL_MOCK1E10_111821_min1000.fa

#actually nevermind this still has header bc assembly has header
#takes first column 
awk '{print $1}' ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000.fa > ARSTAG_CONTROL_MOCK1E10_111821_min1000.fa


In [ ]:
anvi-gen-contigs-database -f /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_min1000.fa \
                          -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_CONTROL_MOCK1E10_111821.idba_ud/ARSTAG_CONTROL_MOCK1E10_111821_scaffold_min1000_contigs.db \
                          -n 'ARSTAG_database' -t 6

#look up the -n option for what goes there
need to download anvio locally



In [ ]:
#replace activate anvio command in the file rose already created 

sed 's/conda activate anvio-7.1;/conda run -n anvio-7.1/' anvio_1-analyze_contigs.sh > anvio_1b-analyze_contigs.sh




run in a tmux with parallel

rose running first anvio commmand
wc -lh anvio_0-fix.sh (there's 15 of them)
                       
cat anvio_0...sh | parallel -j 3

# Cross-mapping

In [18]:
df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})
sample_id = df["sample_id"].tolist()
sample_id


['ARSTAG_TAPRES_TAPRES_23',
 'ARSTAG_AR_1_23',
 'ARSTAG_AR_2_23',
 'ARSTAG_AR_3_23',
 'ARSTAG_AR_4_23',
 'ARSTAG_AR_5_23',
 'ARSTAG_TAPRES_TAPRES_27',
 'ARSTAG_AR_1_27',
 'ARSTAG_AR_2_27',
 'ARSTAG_AR_3_27',
 'ARSTAG_AR_4_27',
 'ARSTAG_AR_5_27',
 'ARSTAG_TAPRES_TAPRES_40',
 'ARSTAG_AR_1_40',
 'ARSTAG_AR_2_40',
 'ARSTAG_AR_3_40',
 'ARSTAG_AR_4_40',
 'ARSTAG_AR_5_40',
 'ARSTAG_TAPRES_TAPRES_41',
 'ARSTAG_AR_1_41',
 'ARSTAG_AR_2_41',
 'ARSTAG_AR_3_41',
 'ARSTAG_AR_4_41',
 'ARSTAG_AR_5_41',
 'ARSTAG_BF_1_40',
 'ARSTAG_BF_2_40',
 'ARSTAG_BF_3_40',
 'ARSTAG_BF_4_40',
 'ARSTAG_BF_5_40',
 'ARSTAG_BF_1_41',
 'ARSTAG_BF_2_41',
 'ARSTAG_BF_3_41',
 'ARSTAG_BF_4_41',
 'ARSTAG_BF_5_41',
 'ARSTAG_CONTROL_BFSLIDECONTROL_41',
 'ARSTAG_CONTROL_MANIFB_41',
 'ARSTAG_CONTROL_MOCK1E10_111821',
 'ARSTAG_CONTROL_MOCK1E8_111821',
 'ARSTAG_AR_5_23',
 'ARSTAG_AR_5_27',
 'ARSTAG_AR_4_40',
 'ARSTAG_BF_1_41',
 'ARSTAG_TAPRES_TAPRES_41']

In [19]:
assembly_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']


In [20]:
with open(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_3-bt2build.sh', 'w') as f:
    for s in assembly_id: 
        scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
        bt2ind = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
        # index in prep for bowtie2 mapping to get coverage on min1000 scaffolds
        f.write(f'bowtie2-build {scaffolds} {bt2ind}\n')

In [21]:
shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio= 'conda run -n anvio-7.1'
reads_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed'
assem_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies'

xmapping_df = []
for s in assembly_id:  

    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
    bt2base = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    
    # map against other samples and controls
    for r in sample_id:
        
        r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
        r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
        raw_bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw.bam'
        filtered_bam_raw = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw-filt.bam'
        bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}.bam'
        
        if 'CONTROL' in r:
            
            # filter mapping - for decontamination, we want to use filtered mappings
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {raw_bam}; \
            reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=2 threads=48; rm {raw_bam}"'
        
        else:
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        # process mapping
        initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; \
        rm {filtered_bam_raw}'
        
         # anvi-profile
        profile_out = f'{assem_dir}/{s}.idba_ud/anvio_data/{r}_profile' # check this
        anvip_cmd = f'sbatch --wrap "anvi-profile --min-contig-length 1000 -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r}"'
        
        ## append all commands to table
        xmapping_df.append([s, r, map_cmd, initbam, anvip_cmd])

xmapping_df_all = pd.DataFrame.from_records(xmapping_df, columns=['assembly', 'reads', 'map', 'initbam', 'anvi-profile'])

#also treat that contaminated control like a sample, only ARs used to make that coassembly



In [22]:
coass_dict = {
    'ARSTAG_ARBF_1_post':["ARSTAG_AR_1_40", "ARSTAG_AR_1_41","ARSTAG_BF_1_40","ARSTAG_BF_1_41"],
    'ARSTAG_ARBF_2_post':["ARSTAG_AR_2_40", "ARSTAG_AR_2_41","ARSTAG_BF_2_40","ARSTAG_BF_2_41"],
    'ARSTAG_ARBF_3_post':["ARSTAG_AR_3_40", "ARSTAG_AR_3_41","ARSTAG_BF_3_40","ARSTAG_BF_3_41"],
    'ARSTAG_ARBF_4_post':["ARSTAG_AR_4_40", "ARSTAG_AR_4_41","ARSTAG_BF_4_40","ARSTAG_BF_4_41"],
    'ARSTAG_ARBF_5_post':["ARSTAG_AR_5_40", "ARSTAG_AR_5_41","ARSTAG_BF_5_40","ARSTAG_BF_5_41"],
    'ARSTAG_ARBF_12345_pre':["ARSTAG_AR_1_23", "ARSTAG_AR_1_27", "ARSTAG_AR_2_23", "ARSTAG_AR_2_27", "ARSTAG_AR_3_23", "ARSTAG_AR_3_27", "ARSTAG_AR_4_23", "ARSTAG_AR_5_23", "ARSTAG_AR_5_27"]}



In [23]:
#now only select the lines that have cross maps we want to keep 

    
# xmapping_df = pd.DataFrame()

# xmapping_df_all.loc[xmapping_df_all['assembly'] == value]

# for row in xmapping_df_all.itertuples():
#     a= row.assembly 
#     s= row.reads
#     if 'TAPRES' in a:
#         if 'TAPRES' in s:
#             print(r)
#             xmapping_df.append(row, ignore_index = True)

xmapping_tapres = xmapping_df_all.loc[(xmapping_df_all['assembly'].str.startswith('ARSTAG_TAPRES')) & (xmapping_df_all['reads'].str.startswith('ARSTAG_TAPRES'))]
len(xmapping_tapres)

xmapping_AR4_27 = xmapping_df_all.loc[(xmapping_df_all['assembly']=='ARSTAG_AR_4_27') & (xmapping_df_all['reads'].str.startswith('ARSTAG_AR_4'))]

xmapping_control = xmapping_df_all.loc[(xmapping_df_all['assembly'].str.startswith('ARSTAG_CONTROL')) & (xmapping_df_all['reads'].str.startswith('ARSTAG_CONTROL'))]
len(xmapping_control)

# xmapping_ARs = xmapping_df_all.loc[(xmapping_df_all['assembly'].str.startswith('ARSTAG_AR') | xmapping_df_all['assembly'].str.startswith('ARSTAG_BF')) & (xmapping_df_all['reads'].str.startswith('ARSTAG_AR') | xmapping_df_all['reads'].str.startswith('ARSTAG_BF'))]
# len(xmapping_ARs)

xmapping_ARBF1 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_1_post') & (xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_1_post"]))]
len(xmapping_ARBF1)
xmapping_ARBF1 #WHY IS THERE AN EXTRA SAMPLE IN HERE, BF_1_41

xmapping_ARBF2 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_2_post') & ((xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_2_post"])) | (xmapping_df_all.reads == 'ARSTAG_CONTROL_BFSLIDECONTROL_41'))]

xmapping_ARBF3 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_3_post') & ((xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_3_post"])) | (xmapping_df_all.reads == 'ARSTAG_CONTROL_BFSLIDECONTROL_41'))]

xmapping_ARBF4 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_4_post') & ((xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_4_post"])) | (xmapping_df_all.reads == 'ARSTAG_CONTROL_BFSLIDECONTROL_41'))]
#extra sample AR_4_40 !

xmapping_ARBF5 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_5_post') & ((xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_5_post"])) | (xmapping_df_all.reads == 'ARSTAG_CONTROL_BFSLIDECONTROL_41'))]


xmapping_ARBF6 = xmapping_df_all.loc[(xmapping_df_all['assembly'] == 'ARSTAG_ARBF_12345_pre') & ((xmapping_df_all.reads.isin(coass_dict["ARSTAG_ARBF_12345_pre"])) | (xmapping_df_all.reads == 'ARSTAG_CONTROL_BFSLIDECONTROL_41'))]


# xmapping_df.head()

In [24]:

xmapping_AR4_27['map'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_xmap4.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping_AR4_27['initbam'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_initbam4.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping_AR4_27['anvi-profile'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_anvip4.sh', index=False, quoting=csv.QUOTE_NONE, header=False)


In [25]:
#combine some of these but don't go longer than 30

xmapping1 = xmapping_tapres.append(xmapping_AR4_27)

xmapping2 = xmapping_ARBF1.append(xmapping_ARBF2)
xmapping2 = xmapping2.append(xmapping_ARBF3)
xmapping2 = xmapping2.append(xmapping_ARBF4)
xmapping2 = xmapping2.append(xmapping_ARBF5)

xmapping3 = xmapping_control.append(xmapping_ARBF6)
len(xmapping3)
len(xmapping1)
len(xmapping2)

26

In [26]:
xmapping1['map'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_xmap1.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping1['initbam'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_initbam1.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping1['anvi-profile'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_anvip1.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

xmapping2['map'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_xmap2.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping2['initbam'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_initbam2.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping2['anvi-profile'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_anvip2.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

xmapping3['map'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_xmap3.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping3['initbam'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_initbam3.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
xmapping3['anvi-profile'].to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_anvip3.sh', index=False, quoting=csv.QUOTE_NONE, header=False)



In [ ]:
# run map (submitted first command for first batch of files but need to resubmit bc hadn't run bt2 build, running again now, all 3 done )

# run init bam (ran in tmux after activating anvio 7.1, so far ran 1st)
tmux
conda activate anvio-7.1
cat anvio_initbam.sh | parallel -j 10

# run anvi profile (running with anvio 7 so not activating conda environment)


## anvi merge

In [ ]:
# anvi-refine manual review
# might be able to use auto binner like concoct

#first migrate to newer anvio 7.1 since the profiles were made in anvio 7.0
anvi-migrate --migrate-dbs-safely *.idba_ud/anvio_data/*_profile/*PROFILE.db #run this with anvio 7.1 activated

anvi-migrate --migrate-dbs-safely ARSTAG_AR_4_27.idba_ud/anvio_data/*_profile/*PROFILE.db

anvi-migrate --migrate-dbs-safely *.idba_ud/anvio_data/merged/*PROFILE.db #run this with anvio 7.1 activated

In [54]:
#write loop for anvi-merge (provide all profiles for that assembly, make new profile in folder called merged)

assembly_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']

cmd = []
for s in assembly_id:
    merge_cmd = f'sbatch --partition memory --wrap "conda run -n anvio-7.1 anvi-merge {assem_dir}/{s}.idba_ud/anvio_data/*_profile/PROFILE.db -o {assem_dir}/{s}.idba_ud/anvio_data/merged -c {assem_dir}/{s}.idba_ud/{s}_contigs.db -S {s}_merged --enforce-hierarchical-clustering"'
    cmd.append(merge_cmd)
    
cmd = pd.Series(cmd, name='cmd')
cmd.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvi-merge.csv', index=False, quoting=csv.QUOTE_NONE, header=False)

#sbatch --partition memory --wrap "conda run -n anvio-7.1 anvi-merge /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/anvio_data/*_profile/PROFILE.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/anvio_data/merged -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/ARSTAG_AR_4_27_contigs.db -S ARSTAG_AR_4_27_merged --enforce-hierarchical-clustering"

# Taxonomy 

In [ ]:
# estimate SCG taxonomy
# must migrate contigs.dbs to 7.1 and run this with 7.1
estscg_taxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/{s}_profile/PROFILE.db'
    out = f'{assem_dir}/{s}.idba_ud/anvio_data/{s}_scg_taxonomy.txt'
    anvestscg = f'anvi-estimate-scg-taxonomy -c {contigsDB} -T 16 --metagenome-mode -o {out} -p {profileDB} --compute-scg-coverages' -p {profileDB} --compute-scg-coverages -S Ribosomal_S2
    estscg_taxonomy_cmd.append(anvestscg)

df['estscg_taxonomy_cmd'] = estscg_taxonomy_cmd

# save files
df['estscg_taxonomy_cmd'].to_csv(f'{outdir}/anvio_estscg_taxonomy_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

In [ ]:
# amplecp 'hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_*/anvio_data/*scg_taxonomy.txt' ./


In [67]:
#ran estimate-scg-taxonomy commands that produced text files for each profile

tax12345 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_12345_pre_scg_taxonomy.txt', sep='\t')
tax5 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_5_post_scg_taxonomy.txt', sep='\t')
tax4 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_4_post_scg_taxonomy.txt', sep='\t')
tax3 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_3_post_scg_taxonomy.txt', sep='\t')
tax2 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_2_post_scg_taxonomy.txt', sep='\t')
tax1 = pd.read_csv('/Users/hanna0hgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_ARBF_1_post_scg_taxonomy.txt', sep='\t')
tax427 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_AR_4_27_scg_taxonomy.txt', sep='\t')
taxCB = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_CONTROL_BFSLIDECONTROL_41_scg_taxonomy.txt', sep='\t')
taxCM = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_CONTROL_MANIFB_41_scg_taxonomy.txt', sep='\t')
taxC10 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_CONTROL_MOCK1E10_111821_scg_taxonomy.txt', sep='\t')
taxC8 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_CONTROL_MOCK1E8_111821_scg_taxonomy.txt', sep='\t')
taxT23 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_TAPRES_TAPRES_23_scg_taxonomy.txt', sep='\t')
taxT27 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_TAPRES_TAPRES_27_scg_taxonomy.txt', sep='\t')
taxT40 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_TAPRES_TAPRES_40_scg_taxonomy.txt', sep='\t')
taxT41 = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/taxonomy_scg/ARSTAG_TAPRES_TAPRES_41_scg_taxonomy.txt', sep='\t')


In [70]:
print(f'AR12345_pre has an estimated {len(tax12345)} genomes')
print(f'AR5_post has an estimated {len(tax5)} genomes')
print(f'AR4_post has an estimated {len(tax4)} genomes')
print(f'AR3_post has an estimated {len(tax3)} genomes')
print(f'AR2_post has an estimated {len(tax2)} genomes')
print(f'AR1_post has an estimated {len(tax1)} genomes')
print(f'AR4_27 has an estimated {len(tax427)} genomes')
print(f'Slide control has an estimated {len(taxCB)} genomes')
print(f'ManifB has an estimated {len(taxCM)} genomes')
print(f'Pos control high has an estimated {len(taxC10)} genomes')
print(f'Pos control low has an estimated {len(taxC8)} genomes')
print(f'tap 23 has an estimated {len(taxT23)} genomes')
print(f'tap 27 has an estimated {len(taxT27)} genomes')
print(f'tap 40 has an estimated {len(taxT40)} genomes')
print(f'tap 41 has an estimated {len(taxT41)} genomes')

AR12345_pre has an estimated 56 genomes
AR5_post has an estimated 51 genomes
AR4_post has an estimated 45 genomes
AR3_post has an estimated 45 genomes
AR2_post has an estimated 47 genomes
AR1_post has an estimated 42 genomes
AR4_27 has an estimated 29 genomes
Slide control has an estimated 31 genomes
ManifB has an estimated 0 genomes
Pos control high has an estimated 8 genomes
Pos control low has an estimated 8 genomes
tap 23 has an estimated 54 genomes
tap 27 has an estimated 23 genomes
tap 40 has an estimated 37 genomes
tap 41 has an estimated 49 genomes


## Make new df

In [8]:
assem_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']

df= pd.DataFrame()

df["assem_id"] = assem_id
df["genome_est"] = [29, 56, 42, 47, 45, 45, 51, 54, 23, 37, 49, 31, 0, 8,8]

df

,assem_id,genome_est
0,ARSTAG_AR_4_27,29
1,ARSTAG_ARBF_12345_pre,56
2,ARSTAG_ARBF_1_post,42
3,ARSTAG_ARBF_2_post,47
4,ARSTAG_ARBF_3_post,45
5,ARSTAG_ARBF_4_post,45
6,ARSTAG_ARBF_5_post,51
7,ARSTAG_TAPRES_TAPRES_23,54
8,ARSTAG_TAPRES_TAPRES_27,23
9,ARSTAG_TAPRES_TAPRES_40,37


## Concoct Command

In [10]:
outfile = f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_concoct.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.assem_id
        c = row.genome_est
        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        out = f'{assem_dir}/{s}.idba_ud/anvio_data'

        get_cococt_input = f'anvi-export-splits-and-coverages -p {profileDB} -c {contigsDB} -o {out} -O {s} --splits-mode'
        run_concoct = f'concoct --coverage_file {out}/{s}-COVs.txt --composition_file {out}/{s}-SPLITS.fa -b {out}/{s}_concoct -r 150 -t 10 -c {c}'
        csv_to_tsv = f'sed "s/,/\\tbin_/g" {out}/{s}_concoct_clustering_gt1000.csv > {out}/{s}_concoct_clustering_gt1000.tsv'
        import_collection = f'anvi-import-collection {out}/{s}_concoct_clustering_gt1000.tsv -p {profileDB} -c {contigsDB} -C concoct'

        f.write(f'{get_cococt_input}; {run_concoct}; {csv_to_tsv}; {import_collection}\n')
        
#had to change the number of estimated genomes in MANIFB_41 from 0 to 1 (in atom)

## Binning

In [13]:
#commands for anvi interactive (ended up using anvi-refine) for each assembly
outfile = f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_interactive.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.assem_id
        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        out = f'{assem_dir}/{s}.idba_ud/anvio_data'

        inter= f'anvi-interactive -p {profileDB} -c {contigsDB} --server-only -P 8080 -C concoct'
        
        f.write(f'{inter}\n')



In [38]:
#commands for anvi-summarize for each assembly
outfile = f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_summarize.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.assem_id

        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        out = f'{assem_dir}/{s}.idba_ud/anvio_data'

        inter= f'anvi-summarize -p {profileDB} -c {contigsDB} -o {out}/{s}_binsummary.html -C concoct'
        
        f.write(f'{inter}\n')

In [39]:
scp hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/*.idba_ud/anvio_data/*_binsummary/index.html /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data/

scp hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_4_27.idba_ud/anvio_data/ARSTAG_AR_4_27_binsummary/index.html /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data/

    

SyntaxError: invalid syntax (<ipython-input-39-c96d8073159f>, line 1)

# Update quality table after mapping

Create code to run on server to get quality metrics

In [35]:
#figure out the number of reads from each sample that mapped to each assembly

mapping_cmd = []
for s,v in assem_dict.items():
    for r in v:
        cmd= f'anvi-export-table {s}.idba_ud/anvio_data/{r}_profile/PROFILE.db --table layer_additional_data -o {s}.idba_ud/anvio_data/{r}_reads_mapped.txt'
        mapping_cmd.append(cmd) 
        
cmd = pd.Series(mapping_cmd, name='cmd')
cmd.to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_mappingQ.sh', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)


In [ ]:
#download files
scp hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/*.idba_ud/anvio_data/*_reads_mapped.txt /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data

    /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_AR_5_40.idba_ud/anvio_data/ARSTAG_AR_5_40_reads_mapped.txt
    

Start running python code here:

In [16]:
#read in files
quality_samp= pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/sample_quality_table_postass.csv', sep=',')
quality_assem= pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/assem_quality_table_postass.csv', sep=',')

quality_samp.head()


,Unnamed: 0,sample_id,raw_file_F,raw_file_R,trim_file_F,trim_file_R,raw_num_seq_F,raw_num_seq_R,trim_num_seq_F,trim_num_seq_R,perc_seq_lost_to_trim,trim_bpsF,trim_bpsR,trim_bps,combined_file,N50,scaffold_count,assembly_size
0,0,ARSTAG_TAPRES_TAPRES_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,38735372,38735372,38616183,38616183,0.307701,5714577182,5717272608,11431849790,/groups/banfield/sequences/2022/ARSTAG_TAPRES_...,1322,554239,511557867
1,1,ARSTAG_AR_1_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,26391193,26391193,26273730,26273730,0.445084,3862453098,3863143875,7725596973,/groups/banfield/sequences/2022/ARSTAG_AR_1_23...,16075,76242,148886568
2,2,ARSTAG_AR_2_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,37543974,37543974,37390867,37390867,0.407807,5488794620,5486098408,10974893028,/groups/banfield/sequences/2022/ARSTAG_AR_2_23...,18682,79234,179057845
3,3,ARSTAG_AR_3_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,42645590,42645590,42449515,42449515,0.459778,6261247154,6263941418,12525188572,/groups/banfield/sequences/2022/ARSTAG_AR_3_23...,18632,98036,184327829
4,4,ARSTAG_AR_4_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,29074701,29074701,28936051,28936051,0.476875,4142238876,4142289753,8284528629,/groups/banfield/sequences/2022/ARSTAG_AR_4_23...,6744,95368,157358641


In [34]:
#for samples, add the total number of reads per sample mapped to an assembly to sample quality table

quality_path = "/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data/"
os.chdir(quality_path)

df= quality_samp
# df = df[(df["sample_id"] != "ARSTAG_AR_5_23") & (df["sample_id"] != "ARSTAG_AR_5_27")] #remove this line once have files

df["total_reads_mapped"]= ""

for row in df.itertuples():
    sample_id = row.sample_id
    r= row.Index
    df_samp = pd.read_csv(f'{quality_path}{sample_id}_reads_mapped.txt', sep='\t')
    total_reads_mapped = df_samp["data_value"][df_samp["data_key"] == "total_reads_mapped"][0]
    df.loc[r,'total_reads_mapped'] = total_reads_mapped

    
df.loc[:,"total_reads_mapped"]= pd.to_numeric(df["total_reads_mapped"])

quality_samp = df


In [35]:
#calculate percent mapped and add to sample quality table

quality_samp["percent_reads_mapped"] = quality_samp["total_reads_mapped"]/(quality_samp["trim_num_seq_F"]+quality_samp["trim_num_seq_R"])*100
quality_samp

,Unnamed: 0,sample_id,raw_file_F,raw_file_R,trim_file_F,trim_file_R,raw_num_seq_F,raw_num_seq_R,trim_num_seq_F,trim_num_seq_R,perc_seq_lost_to_trim,trim_bpsF,trim_bpsR,trim_bps,combined_file,N50,scaffold_count,assembly_size,total_reads_mapped,percent_reads_mapped
0,0,ARSTAG_TAPRES_TAPRES_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,38735372,38735372,38616183,38616183,0.307701,5714577182,5717272608,11431849790,/groups/banfield/sequences/2022/ARSTAG_TAPRES_...,1322,554239,511557867,50018088,64.763118
1,1,ARSTAG_AR_1_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,26391193,26391193,26273730,26273730,0.445084,3862453098,3863143875,7725596973,/groups/banfield/sequences/2022/ARSTAG_AR_1_23...,16075,76242,148886568,48468409,92.237396
2,2,ARSTAG_AR_2_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,37543974,37543974,37390867,37390867,0.407807,5488794620,5486098408,10974893028,/groups/banfield/sequences/2022/ARSTAG_AR_2_23...,18682,79234,179057845,68649513,91.799841
3,3,ARSTAG_AR_3_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,42645590,42645590,42449515,42449515,0.459778,6261247154,6263941418,12525188572,/groups/banfield/sequences/2022/ARSTAG_AR_3_23...,18632,98036,184327829,79049647,93.110189
4,4,ARSTAG_AR_4_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,29074701,29074701,28936051,28936051,0.476875,4142238876,4142289753,8284528629,/groups/banfield/sequences/2022/ARSTAG_AR_4_23...,6744,95368,157358641,50675474,87.564599
5,5,ARSTAG_AR_5_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,28011485,28011485,27871385,27871385,0.500152,4046405618,4048514411,8094920029,/groups/banfield/sequences/2022/ARSTAG_AR_5_23...,6748,108426,185939560,50025362,89.743229
6,6,ARSTAG_TAPRES_TAPRES_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,33761415,33761415,33603683,33603683,0.467196,4906022542,4911716747,9817739289,/groups/banfield/sequences/2022/ARSTAG_TAPRES_...,878,284917,216044543,32827887,48.845668
7,7,ARSTAG_AR_1_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,27493842,27493842,27377695,27377695,0.422447,4039802210,4041261360,8081063570,/groups/banfield/sequences/2022/ARSTAG_AR_1_27...,10576,78893,153614428,50832749,92.836064
8,8,ARSTAG_AR_2_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,34751616,34751616,34581359,34581359,0.489925,5123709949,5113616600,10237326549,/groups/banfield/sequences/2022/ARSTAG_AR_2_27...,10983,102348,186453734,59617924,86.199510
9,9,ARSTAG_AR_3_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,5024

In [36]:
#save new versions of quality table

quality_samp.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/sample_quality_table_postmap.csv')
# df.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/assem_quality_table_postass.csv')


In [35]:
#begin adding to assembly quality table
# assem_dict
# #add binning stats
# quality_assem["samps_in_assem"] = assem_dict
quality_assem
assem_dict
# quality_assem.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/figures/assembly_quality_table.csv')


{'ARSTAG_ARBF_1_post': ['ARSTAG_AR_1_40',
  'ARSTAG_AR_1_41',
  'ARSTAG_BF_1_40',
  'ARSTAG_BF_1_41'],
 'ARSTAG_ARBF_2_post': ['ARSTAG_AR_2_40',
  'ARSTAG_AR_2_41',
  'ARSTAG_BF_2_40',
  'ARSTAG_BF_2_41'],
 'ARSTAG_ARBF_3_post': ['ARSTAG_AR_3_40',
  'ARSTAG_AR_3_41',
  'ARSTAG_BF_3_40',
  'ARSTAG_BF_3_41'],
 'ARSTAG_ARBF_4_post': ['ARSTAG_AR_4_40',
  'ARSTAG_AR_4_41',
  'ARSTAG_BF_4_40',
  'ARSTAG_BF_4_41'],
 'ARSTAG_ARBF_5_post': ['ARSTAG_AR_5_40',
  'ARSTAG_AR_5_41',
  'ARSTAG_BF_5_40',
  'ARSTAG_BF_5_41'],
 'ARSTAG_ARBF_12345_pre': ['ARSTAG_AR_1_23',
  'ARSTAG_AR_1_27',
  'ARSTAG_AR_2_23',
  'ARSTAG_AR_2_27',
  'ARSTAG_AR_3_23',
  'ARSTAG_AR_3_27',
  'ARSTAG_AR_4_23',
  'ARSTAG_AR_5_23',
  'ARSTAG_AR_5_27'],
 'ARSTAG_AR_4_27': ['ARSTAG_AR_4_27'],
 'ARSTAG_TAPRES_TAPRES_23': ['ARSTAG_TAPRES_TAPRES_23'],
 'ARSTAG_TAPRES_TAPRES_27': ['ARSTAG_TAPRES_TAPRES_27'],
 'ARSTAG_TAPRES_TAPRES_40': ['ARSTAG_TAPRES_TAPRES_40'],
 'ARSTAG_TAPRES_TAPRES_41': ['ARSTAG_TAPRES_TAPRES_41'],
 'ARSTAG_CON

## Fixing missing profiles in ARSTAG_ARBF_12345_pre

In [ ]:
#Somehow AR_5_23 and AR_5_27 weren't included in the merged ARBF_12345_pre profile

#Step 1: export bins from ARBF_12345_pre to a txt file
anvi-export-collection -C concoct -p PROFILE.db
#created /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/merged/collection-concoct-info.txt 

# Step 2: run mapping and profiling to make those missing individual profiles

sbatch --wrap "/shared/software/bin/bowtie2 -p 48 -x /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/bt2/ARSTAG_ARBF_12345_pre_min1000.fa -1 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_5_23_trim_clean.PE.1.fastq.gz -2 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_5_23_trim_clean.PE.2.fastq.gz | /shared/software/bin/shrinksam -v | sambam > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_23-raw-filt.bam"
sbatch --wrap "/shared/software/bin/bowtie2 -p 48 -x /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/bt2/ARSTAG_ARBF_12345_pre_min1000.fa -1 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_5_27_trim_clean.PE.1.fastq.gz -2 /groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed/ARSTAG_AR_5_27_trim_clean.PE.2.fastq.gz | /shared/software/bin/shrinksam -v | sambam > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_27-raw-filt.bam"

    #tmux; conda activate anvio-7.1
anvi-init-bam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_23-raw-filt.bam -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_23.bam;         rm /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_23-raw-filt.bam
anvi-init-bam /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_27-raw-filt.bam -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_27.bam;         rm /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_27-raw-filt.bam

    #anvio 7.0
sbatch --wrap "anvi-profile --min-contig-length 1000 -T 48 -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_23.bam -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_contigs.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_23_profile -S ARSTAG_AR_5_23"
sbatch --wrap "anvi-profile --min-contig-length 1000 -T 48 -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre-vs-ARSTAG_AR_5_27.bam -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_contigs.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_27_profile -S ARSTAG_AR_5_27"

#Step 3: merge all individual profiles into a new merged profile

    #migrate profile to anvio 7.1 #run this with anvio 7.1 activated
anvi-migrate --migrate-dbs-safely ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_23_profile/PROFILE.db 
anvi-migrate --migrate-dbs-safely ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_27_profile/PROFILE.db 
anvi-migrate --migrate-dbs-safely ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/*_profile/PROFILE.db 

    #merge into folder merged
sbatch --partition memory --wrap "conda run -n anvio-7.1 anvi-merge /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/*_profile/PROFILE.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/merged -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_contigs.db -S ARSTAG_ARBF_12345_pre_merged --enforce-hierarchical-clustering"

    #get mapping quality txt file (then download these locally)
anvi-export-table ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_23_profile/PROFILE.db --table layer_additional_data -o ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_23_reads_mapped.txt
anvi-export-table ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_27_profile/PROFILE.db --table layer_additional_data -o ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_27_reads_mapped.txt
scp hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_23_reads_mapped.txt /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data
scp hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/ARSTAG_AR_5_27_reads_mapped.txt /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data


#Step 4: import your original bins to the new ARBF_12345_pre
scp /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_data/collection-concoct.tsv hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/merged_no5/ 

anvi-import-collection -C /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/merged_no5/collection-concoct.txt -p PROFILE.db -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_contigs.db
anvi-import-collection -C /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/anvio_data/merged_no5/collection-concoct.tsv -p PROFILE.db -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_12345_pre.idba_ud/ARSTAG_ARBF_12345_pre_contigs.db


# Dereplicating Bins

In [18]:
#locally download binning summary output files from anvi-summarize which has binning information for each bin in each assembly
/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_1_post.idba_ud/anvio_data/binning-final/bin_by_bin/bin_11/bin_11-contigs.fa

/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/*.idba_ud/anvio_data/binning-final/bin_by_bin/bin_*/*-contigs.fa

#then make individual csv files with output table for each assembly
#add column with assembly base name (next section add in bin name and .fa after assembly name)

SyntaxError: invalid syntax (3896903946.py, line 4)

In [19]:
#create table with commands and paths needed for drep

assembly_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']

#construct a table with paths to files, assembly names, and bin names 
#then add new column with new path and bin names with assembly in front of it
#then make a command column cmd=f’cp {from} {to}’ to change names and put in centralized folder

from glob import glob
files = glob('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/binning_summary/*.csv')
drep_table = pd.DataFrame()
drep_table["path_to_bin"]= ""
drep_table["mv_cmd"]= ""
drep_table["new_path_to_bin"]= ""
for f in files:
    df = pd.read_csv(f)
    for row in df.itertuples():
        r= row.Index
        a= row.Assembly
        b= row.Bin
        path_to_bin= f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/{a}.idba_ud/anvio_data/binning-final/bin_by_bin/{row.Bin}/{row.Bin}-contigs.fa'
        new_path_to_bin= f'/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep-genomes'
        df.loc[r, 'path_to_bin'] = path_to_bin
        mv_cmd = f'cp {path_to_bin} {new_path_to_bin}/{a}_{b}.fa'
        df.loc[r, 'genome']= f'{a}_{b}'
#         df.loc[r, 'completeness']= row.
#         df.loc[r, 'contamination']= {a}_{b}
        df.loc[r, 'mv_cmd'] = mv_cmd
    drep_table = drep_table.append(df)

drep_table= drep_table.rename(columns={"Compl.": "completeness", "Red.": "contamination"})
drep_table['completeness'] = drep_table['completeness'].str.rstrip('%').astype('float')
drep_table['contamination'] = drep_table['contamination'].str.rstrip('%').astype('float')
drep_table['genome'] = drep_table['genome']+ ".fa"
genome_info = drep_table.drop(columns =["N50", "actual mock", "mv_cmd", "new_path_to_bin", "Bin", "Source", "Taxonomy", "Total Size", "Num Contigs", "GC Content", "path_to_bin", "SCG Domain", "Assembly"])

genome_info


/var/folders/20/cc4365f176jg1_x05lhm6g080000gn/T/ipykernel_41746/1440177914.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drep_table = drep_table.append(df)
/var/folders/20/cc4365f176jg1_x05lhm6g080000gn/T/ipykernel_41746/1440177914.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drep_table = drep_table.append(df)
/var/folders/20/cc4365f176jg1_x05lhm6g080000gn/T/ipykernel_41746/1440177914.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drep_table = drep_table.append(df)
/var/folders/20/cc4365f176jg1_x05lhm6g080000gn/T/ipykernel_41746/1440177914.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drep_table = drep_table

,completeness,contamination,genome
0,100.00,1.41,ARSTAG_AR_4_27_bin_26_1.fa
1,100.00,9.86,ARSTAG_AR_4_27_bin_unknown_2.fa
2,100.00,0.00,ARSTAG_AR_4_27_bin_5_5.fa
3,100.00,0.00,ARSTAG_AR_4_27_bin_23_1.fa
4,100.00,2.82,ARSTAG_AR_4_27_bin_26_5.fa
...,...,...,...
5,61.97,0.00,ARSTAG_CONTROL_MOCK1E8_111821_bin_3.fa
6,57.75,7.04,ARSTAG_CONTROL_MOCK1E8_111821_bin_8.fa
7,53.52,1.41,ARSTAG_CONTROL_MOCK1E8_111821_bin_7.fa
8,30.26,3.95,ARSTAG_CONTROL_MOCK1E8_111821_bin_5.fa


In [8]:
# write the commands to a CSV
drep_table['mv_cmd'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/mv-before-drep.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
genome_info.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/genome_info.csv', index=False, quoting=csv.QUOTE_NONE, header= True)



In [ ]:
# move genome_info into biotite
scp /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/genome_info.csv hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/


In [ ]:
# write drep commands using table we created 
## drep: https://drep.readthedocs.io/en/latest/overview.html# 
## drep commands: https://drep.readthedocs.io/en/latest/module_descriptions.html 

## GENOMEINFO: location of .csv file containing quality information on the genomes. Must contain: ["genome"(basename of.fasta file of that genome), "completeness"(0-100 value for completeness of the genome), "contamination"(0-100 value of
## GENOMES: , --genomes [GENOMES [GENOMES ...]]
#                         genomes to filter in .fasta format. Not necessary if
#                         Bdb or Wdb already exist. Can also input a text file
#                         with paths to genomes, which results in fewer OS
#                         issues than wildcard expansion (default: None)
#end with work directory

sbatch --wrap "dRep dereplicate /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep --debug -comp 90 -con 10 --genomeInfo genome_info.csv --checkM_method lineage_wf -g drep-genomes/*.fa" #debug option just gives additional output

sbatch --wrap "dRep dereplicate /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep -p 48 -comp 90 -con 10 --genomeInfo genome_info.csv --checkM_method lineage_wf -g drep-genomes/*.fa"
 


## Mapping Reads to Dereplicated Bins

1. concatenate fasta files of dereplicated bins
2. create bt2 indices. map reads to fasta file. (is this step 2 or 3?) to generate sample BAM files
3. generate contigs.db using the concatenated fasta file (as if this is a coassembly)
4. follow anvio metagenomics workflow to create sample profiles using BAM files

status: ran bt2-build and anvi-gen-contigs-database

In [ ]:
# #read in names of dereplicated genome files
# /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/drep-figures/Wdb.csv



In [ ]:
#concatenate fa files together and put into new folder "MAG_analysis"

cat /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/*.fa > dereplicated-genomes_scaffold.fa



In [ ]:
#make contigs.db (look at other anvio commands in .sh files)

sbatch --wrap "conda activate anvio-7.1; anvi-gen-contigs-database -f /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/dereplicated-genomes_scaffold.fa -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -n 'derep_MAG_database' -T 48"
    #this did NOT activate anvio-7.1 correctly, need init
    #migrate contigs db over
    
    

In [ ]:
#prep for mapping

# make directory to store bowtie2 indices in
mkdir /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/bt2/
    
# index in prep for bowtie2 mapping
sbatch --wrap "bowtie2-build /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/dereplicated-genomes_scaffold.fa /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/bt2/dereplicated-genomes_scaffold.fa" 




In [14]:
bt2 = '/shared/software/bin/bowtie2'
shrinksam= '/shared/software/bin/shrinksam'

reads_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed'
coass_dict = {
    'ARSTAG_ARBF_1_post':["ARSTAG_AR_1_40", "ARSTAG_AR_1_41","ARSTAG_BF_1_40","ARSTAG_BF_1_41"],
    'ARSTAG_ARBF_2_post':["ARSTAG_AR_2_40", "ARSTAG_AR_2_41","ARSTAG_BF_2_40","ARSTAG_BF_2_41"],
    'ARSTAG_ARBF_3_post':["ARSTAG_AR_3_40", "ARSTAG_AR_3_41","ARSTAG_BF_3_40","ARSTAG_BF_3_41"],
    'ARSTAG_ARBF_4_post':["ARSTAG_AR_4_40", "ARSTAG_AR_4_41","ARSTAG_BF_4_40","ARSTAG_BF_4_41"],
    'ARSTAG_ARBF_5_post':["ARSTAG_AR_5_40", "ARSTAG_AR_5_41","ARSTAG_BF_5_40","ARSTAG_BF_5_41"],
    'ARSTAG_ARBF_12345_pre':["ARSTAG_AR_1_23", "ARSTAG_AR_1_27", "ARSTAG_AR_2_23", "ARSTAG_AR_2_27", "ARSTAG_AR_3_23", "ARSTAG_AR_3_27", "ARSTAG_AR_4_23", "ARSTAG_AR_5_23", "ARSTAG_AR_5_27"]}

# sample_id = [ARSTAG_TAPRES_TAPRES_23, ARSTAG_AR_1_23, ARSTAG_AR_2_23, ARSTAG_AR_3_23, ARSTAG_AR_4_23, ARSTAG_AR_5_23, 
#              ARSTAG_TAPRES_TAPRES_27, ARSTAG_AR_1_27,ARSTAG_AR_2_27,ARSTAG_AR_3_27,ARSTAG_AR_4_27,ARSTAG_AR_5_27,
#              ARSTAG_TAPRES_TAPRES_40, ARSTAG_AR_1_40, ARSTAG_AR_2_40, ARSTAG_AR_3_40, ARSTAG_AR_4_40, ARSTAG_AR_5_40,
#              ARSTAG_BF_1_40, ARSTAG_BF_2_40, ARSTAG_BF_3_40, ARSTAG_BF_4_40, ARSTAG_BF_5_40,
#              ARSTAG_TAPRES_TAPRES_41, ARSTAG_AR_1_41, ARSTAG_AR_2_41, ARSTAG_AR_3_41, ARSTAG_AR_4_41, ARSTAG_AR_5_41,
#              ARSTAG_BF_1_41,ARSTAG_BF_2_41, ARSTAG_BF_3_41, ARSTAG_BF_4_41, ARSTAG_BF_5_41,
#              ARSTAG_CONTROL_BFSLIDECONTROL_41, ARSTAG_CONTROL_MANIFB_41, ARSTAG_CONTROL_MOCK1E10_111821,ARSTAG_CONTROL_MOCK1E8_111821]

df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})



In [1]:
#mapping reads to dereplicated genomes
#need to filter to only allow certain number of mismatches (this is -N)

shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio= 'conda run -n anvio-7.1'
scaffold = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/bt2/dereplicated-genomes_scaffold.fa' 
mapping_cmd = []

f_reads = []
r_reads = []
for row in df.itertuples():
    r = row.sample_id
    f_reads.append(f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz')
    r_reads.append(f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz')
f_reads = ','.join(f_reads)
r_reads = ','.join(r_reads)

cmd = f'sbatch --wrap "' \
        f'{bt2} ' \
        f'-p 48 -x {scaffold} ' \
        f'-1 {f_reads} -2 {r_reads} ' \
        f'2> /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_scaffold_mapped.log ' \
        f'| {shrinksam} -v > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_scaffold_mapped.bam;' \
        f'reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=1 threads=48; rm {raw_bam}"'

# process mapping
        initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; \
        rm {filtered_bam_raw}'
        
         # anvi-profile
        profile_out = f'{assem_dir}/{s}.idba_ud/anvio_data/{r}_profile' # check this
        anvip_cmd = f'sbatch --wrap "anvi-profile --min-contig-length 1000 -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r}"'
        
        ## append all commands to table
        xmapping_df.append([s, r, map_cmd, initbam, anvip_cmd])

 

# print(cmd)


IndentationError: unexpected indent (3133260312.py, line 28)

In [26]:
#mapping to dereplicated MAGs

#ran mapping command, then ran initbam, then ran anvip

shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio= 'conda run -n anvio-7.1'
bt2base = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/bt2/dereplicated-genomes_scaffold.fa' 
assem_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/'
reads_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/reads/trimmed'
df = pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/quality_summary.tsv', sep='\t')
df = df.rename(columns={'sample':'sample_id'})
s= 'drepMAGs'
contigsDB = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db'

xmapping_df = []
for row in df.itertuples():
    r = row.sample_id
    r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
    r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
    raw_bam = f'{assem_dir}/{s}-vs-{r}-raw.bam'
    filtered_bam_raw = f'{assem_dir}/{s}-vs-{r}-raw-filt.bam'
    bam = f'{assem_dir}/{s}-vs-{r}.bam'

    map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {raw_bam}; \
    reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=1 threads=48; rm {raw_bam}"'

    # process mapping
    initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; \
    rm {filtered_bam_raw}'

     # anvi-profile
    profile_out = f'{assem_dir}/anvio_data/{r}_profile' 
    anvip_cmd = f'sbatch --wrap "anvi-profile --min-contig-length 1000 -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r}"'

    ## append all commands to table
    xmapping_df.append([s, r, map_cmd, initbam, anvip_cmd])

xmapping_df_all = pd.DataFrame.from_records(xmapping_df, columns=['assembly', 'reads', 'map', 'initbam', 'anvi-profile'])

xmapping_df_all['map'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/drep_mapping.sh', index= False, header= False, quoting=csv.QUOTE_NONE)
xmapping_df_all['initbam'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/drep_initbam.sh', index= False, header= False, quoting=csv.QUOTE_NONE)
xmapping_df_all['anvi-profile'].to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/drep_anvip.sh', index= False, header= False, quoting=csv.QUOTE_NONE)


In [ ]:
# for coassemblies after mapping (remove .sam files that take up a ton of space)

mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/*.sh /groups/banfield/projects/industrial/nelson_lab/arstagnation/workflow/run/
mv /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/slurm* /groups/banfield/projects/industrial/nelson_lab/arstagnation/workflow/slurms/

# rm dereplicated-genomes_scaffold_mapped.sam



In [ ]:
# create binning_results file before import

# The file format for binning_results.txt is very simple. 
# This is supposed to be a TAB-delimited file that contains information about which contig belongs to what bin. 
# So each line of this TAB-delimited file should contain a contig name (or split name, see below), and the bin name it belongs to.
# our binning_results.txt contains split names. 
# But if you have contig names, you can import them using anvi-import-collection with the flag --contigs-mode.

# /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/ARSTAG_ARBF_2_post.idba_ud/anvio_data/merged/

#Rose ended up doing this in her python mentoring notebook. 


In [ ]:
#first migrate to newer anvio 7.1 since the profiles were made in anvio 7.0
anvi-migrate --migrate-dbs-safely *_profile/PROFILE.db #run this with anvio 7.1 activated

#merge all dereplicated profiles into one profile
sbatch --partition memory --wrap "conda run -n anvio-7.1 anvi-merge /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/*_profile/PROFILE.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -S drepgenomes_merged --enforce-hierarchical-clustering"

#import bins to new merged profile
anvi-import-collection -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -C dRep --contigs-mode /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/drep_contigs_to_bin.tsv
    
#need to drop header of tsv
sed -i.bak -e 1d /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/drep_contigs_to_bin.tsv

#collection dRep has been imported to merged profile! 30,203 splits in 90 bins


In [ ]:
# annotate, estimate taxonomy
#commands run on anvio contigs.db
sbatch --wrap "conda run -n anvio-7.1 anvi-get-sequences-for-gene-calls -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_gene_calls.fa; anvi-run-hmms -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -T 48; anvi-run-scg-taxonomy -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -T 48; /groups/banfield/projects/industrial/nelson_lab/kaiju/bin/kaiju -t /shared/db/kaiju/nr_euk/r2021-02-24/nodes.dmp -f /shared/db/kaiju/nr_euk/r2021-02-24/kaiju_db_nr_euk.fmi -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_gene_calls.fa -v -z 48 > /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_kaiju.out"

/groups/banfield/projects/industrial/nelson_lab/kaiju/bin/kaiju-addTaxonNames -t /shared/db/kaiju/nr_euk/r2021-02-24/nodes.dmp -n /shared/db/kaiju/nr_euk/r2021-02-24/names.dmp -r superkingdom,phylum,order,class,family,genus,species -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_kaiju.out -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_genes_kaiju.txt; anvi-import-taxonomy-for-genes -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/drepgenomes_genes_kaiju.txt -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -p kaiju --just-do-it





In [ ]:
# export coverage data (jk this isn't useful because no taxonomy. this is for if you want to bin outside anvio)

anvi-export-splits-and-coverages -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db 

#downloaded coverages file as drepgenomes_merged-COVs.txt, coverage of each split


In [ ]:
# export bin taxonomy and coverage
anvi-summarize -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -o SAMPLES-SUMMARY -C dRep

#coverage of each bin across each sample found in /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/SAMPLES-SUMMARY/bins_across_samples/mean_coverage.txt


In [ ]:
# metabolism
anvi-setup-kegg-kofams

sbatch --wrap "conda run -n anvio-7.1 anvi-run-kegg-kofams -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -T 48"

# anvi-run-kegg-kofams -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db 
anvi-estimate-metabolism -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db -C dRep


## Figuring out which bins are eukaryotes

In [23]:
len(drep_table.Bin.unique())

345

In [47]:
# drep_table[drep_table["SCG Domain"] == "eukarya"]
# drep_table[(drep_table["SCG Domain"] == "eukarya") | (drep_table["Taxonomy"] == "NaN" )]
euk = drep_table[ (drep_table["Taxonomy"]).isna()]
euk = euk[~(euk['Total Size'].str.contains('K'))]
euk['Total Size'] = euk['Total Size'].str.rstrip('Mb').astype('float')
euk = euk[euk['Total Size'] > 5]
# drep_table.head()
# drep_table["SCG Domain"].unique()

euk.to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_euk.csv', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)
euk

#Have 9 bins that label as eukaryotes and aren't controls (same 9 as when taxonomy is unknown and above 5 Mb)
#Of these 9, there are probably 3-4 different genomes based on GC content and size

,path_to_bin,mv_cmd,new_path_to_bin,Bin,Source,Taxonomy,Total Size,Num Contigs,N50,GC Content,completeness,contamination,SCG Domain,Assembly,genome,actual mock
35,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_2,UNKNOWN,NaN,5.15,"2,449","2,296",59.04%,51.81,6.02,eukarya,ARSTAG_AR_4_27,ARSTAG_AR_4_27_bin_2.fa,NaN
40,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_27,UNKNOWN,NaN,6.93,"4,116","1,713",59.50%,26.51,0.00,eukarya,ARSTAG_AR_4_27,ARSTAG_AR_4_27_bin_27.fa,NaN
46,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_37,UNKNOWN,NaN,7.44,"4,659","1,608",51.10%,36.14,3.61,eukarya,ARSTAG_ARBF_4_post,ARSTAG_ARBF_4_post_bin_37.fa,NaN
47,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_6,UNKNOWN,NaN,15.19,"4,680","3,540",52.54%,34.94,2.41,eukarya,ARSTAG_ARBF_4_post,ARSTAG_ARBF_4_post_bin_6.fa,NaN
38,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_22,UNKNOWN,NaN,14.91,"4,326","3,822",53.70%,67.47,7.23,eukarya,ARSTAG_ARBF_12345_pre,ARSTAG_ARBF_12345_pre_bin_22.fa,NaN
65,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_27,UNKNOWN,NaN,6.15,"3,980","1,528",52.12%,13.25,0.00,eukarya,ARSTAG_ARBF_12345_pre,ARSTAG_ARBF_12345_pre_bin_27.fa,NaN
4,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_7,anvi-merge-bins,NaN,10.71,"2,432","5,942",38.21%,83.13,8.43,eukarya,ARSTAG_CONTROL_MOCK1E10_111821,ARSTAG_CONTROL_MOCK1E10_111821_bin_7.fa,NaN
6,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_4,anvi-merge-bins,NaN,9.93,"5,657","1,804",48.18%,16.87,4.82,eukarya,ARSTAG_CONTROL_MOCK1E10_111821,ARSTAG_CONTROL_MOCK1E10_111821_bin_4.fa,NaN
33,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_16,UNKNOWN,NaN,7.42,"1,073","9,727",58.82%,71.08,4.82,eukarya,ARSTAG_ARBF_3_post,ARSTAG_ARBF_3_post_bin_16.fa,NaN
4,/groups/banfield/projects/industrial/nelson_la...,cp /groups/banfield/projects/industrial/nelson...,NaN,bin_7,anvi-merge-bins,NaN,10.71,"2,432","5,942",38.21%,83.13,8.43,eukarya,ARSTAG_CONTROL_MOCK1E10,ARSTAG_CONTROL_MOCK1E10_bin_7.fa,NaN


In [44]:
#create loop for all samples to get 18S

assem_analysis_dir = '/groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/assembly_analysis/'

df_18S = []
# for row in df.itertuples():
     # r = row.sample_id
for s in assembly_id:  
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
   
    cmd_18S = f'anvi-get-sequences-for-hmm-hits -c {contigsDB} --hmm-sources Ribosomal_RNA_18S -o {assem_analysis_dir}/{s}_sequences-for-18S-hmm-hits.fa'
    df_18S.append(cmd_18S) 
        
cmd = pd.Series(df_18S, name='cmd')
cmd.to_csv(f'/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_18S.sh', index=False, quoting=csv.QUOTE_NONE, sep='\t', header=False)



In [45]:
scp /Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/anvio_18S.sh hannahg@biotite.berkeley.edu:/groups/banfield/projects/industrial/nelson_lab/arstagnation/workflow/

In [ ]:
# then concatenated the 18S.fa files produced into all_18S.fa
# then downloaded locally and uploaded to NCBI BLAST

# Add to quality table after dereplicating bins

In [11]:
#read in quality table of samples
quality_samp= pd.read_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/sample_quality_table_postmap.csv', sep=',')

quality_samp= quality_samp.drop([38,39,40,41,42])
quality_samp= quality_samp.drop(columns= ['Unnamed: 0.1', 'Unnamed: 0'])

# quality_samp.columns
quality_samp= quality_samp.sort_values(by= 'sample_id')


In [12]:
#Add a column (sorted alphabetically) with the total reads mapped to the dereplicated genome from each sample
                                                
total_reads_mapped = [36191474, 37276741, 56106457, 72486165, 58250954, 53040286,
                                                49046712, 73898639, 63803481,
                                                62787469, 66511468, 59428729,
                                                45358005, 89422464, 83955726,
                                                58253297, 37501794, 46472776,
                                                69198286, 67859506, 74151345,
                                                53881379, 42811903, 45606742,
                                                46495305, 51906860, 33612429,
                                                53976212, 51673150, 58915616,
                                                68881006, 98071, 37986476,
                                                39078170, 35974421, 30360438,
                                                58982049, 45719452]
quality_samp["total_reads_mapped_to_drep"] = total_reads_mapped
quality_samp["percent_reads_mapped_to_drep_genomes"] = quality_samp["total_reads_mapped_to_drep"]/(quality_samp["trim_num_seq_F"]+quality_samp["trim_num_seq_R"])*100
quality_samp



,sample_id,raw_file_F,raw_file_R,trim_file_F,trim_file_R,raw_num_seq_F,raw_num_seq_R,trim_num_seq_F,trim_num_seq_R,perc_seq_lost_to_trim,...,trim_bpsR,trim_bps,combined_file,N50,scaffold_count,assembly_size,total_reads_mapped,percent_reads_mapped,total_reads_mapped_to_drep,percent_reads_mapped_to_drep_genomes
1,ARSTAG_AR_1_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,26391193,26391193,26273730,26273730,0.445084,...,3863143875,7725596973,/groups/banfield/sequences/2022/ARSTAG_AR_1_23...,16075,76242,148886568,48468409,92.237396,36191474,68.873879
7,ARSTAG_AR_1_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,27493842,27493842,27377695,27377695,0.422447,...,4041261360,8081063570,/groups/banfield/sequences/2022/ARSTAG_AR_1_27...,10576,78893,153614428,50832749,92.836064,37276741,68.078670
13,ARSTAG_AR_1_40,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,32573758,32573758,32362661,32362661,0.648058,...,4795505100,9602280380,/groups/banfield/sequences/2022/ARSTAG_AR_1_40...,10862,83621,166878080,62510558,96.578211,56106457,86.683936
19,ARSTAG_AR_1_41,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,45065764,45065764,44845339,44845339,0.489119,...,6637441524,13283923904,/groups/banfield/sequences/2022/ARSTAG_AR_1_41...,7788,128121,232612645,84662268,94.393609,72486165,80.817947
2,ARSTAG_AR_2_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,37543974,37543974,37390867,37390867,0.407807,...,5486098408,10974893028,/groups/banfield/sequences/2022/ARSTAG_AR_2_23...,18682,79234,179057845,68649513,91.799841,58250954,77.894629
8,ARSTAG_AR_2_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,34751616,34751616,34581359,34581359,0.489925,...,5113616600,10237326549,/groups/banfield/sequences/2022/ARSTAG_AR_2_27...,10983,102348,186453734,59617924,86.199510,53040286,76.689129
14,ARSTAG_AR_2_40,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,29552650,29552650,29308082,29308082,0.827567,...,4350061008,8700959514,/groups/banfield/sequences/2022/ARSTAG_AR_2_40...,42946,70093,159648051,56842459,96.974034,49046712,83.674380
20,ARSTAG_AR_2_41,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,43923976,43923976,43649070,43649070,0.625868,...,6461826038,12927382637,/groups/banfield/sequences/2022/ARSTAG_AR_2_41...,28897,81784,178033529,85287824,97.697183,73898639,84.650875
3,ARSTAG_AR_3_23,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/industrial/nelson_la...,42645590,42645590,42449515,42449515,0.459778,...,6263941418,12525188572,/groups/banfield/sequences/2022/ARSTAG_AR_3_23...,18632,98036,184327829,79049647,93.110189,63803481,75.152191
9,ARSTAG_AR_3_27,/groups/banfield/projects/industrial/nelson_la...,/groups/banfield/projects/indu

In [14]:
#write commands to CSV
# quality_samp.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/sample_quality_table_postderep.csv')
quality_samp.to_csv('/Users/hannahgreenwald/Documents/Documents/Berkeley_Research/AR_metagenomics/figures/sample_quality_table_postderep.csv')


## Try to find taxonomy of unclassified bin

In [ ]:
# #see if 16S is there
anvi-get-sequences-for-hmm-hits -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db -C dRep --list-available-gene-names

# #pull the fasta files
# anvi-get-sequences-for-hmm-hits -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db \
#                                 -p /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/anvio_data/merged/PROFILE.db \
#                                 -C dRep
#                                 --hmm-source Bacteria_71 \
#                                 --gene-names amoA? \
#                                 -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/assembly_analysis/{s}_sequences-for-16S-hmm-hits.fa
#pull the fasta files
anvi-get-sequences-for-hmm-hits -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/workflow/internal_genomes.txt --hmm-source Ribosomal_RNA_16S -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/assembly_analysis/ARSTAG_ARBF_2_post_bin_29_1_sequences-for-16S-hmm-hits.fa

#pull the fasta files for 23S bc couldn't find any 16S
anvi-get-sequences-for-hmm-hits -i /groups/banfield/projects/industrial/nelson_lab/arstagnation/workflow/internal_genomes.txt --hmm-source Ribosomal_RNA_23S 


In [ ]:
#copy unknown bin into new folder
cp /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated_genomes/ARSTAG_ARBF_2_post_bin_29_1.fa /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/gtdb_test/

#https://ecogenomics.github.io/GTDBTk/commands/classify_wf.html
gtdbtk classify_wf --genome_dir /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/gtdb_test/ --out_dir /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/gtdb_test/ -x .fasta --cpus 6


# AmoA sequences

In [ ]:

anvi-get-sequences-for-gene-calls -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db --get-aa-sequences -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/drep_proteins.faa

* so you can feed that ^ output into hmmsearch. 

hmmsearch --cpu 6 --tblout /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/dRep_proteins-vs-amoA.out /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/K10944.hmm /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/drep_proteins.faa

* Then look at the output, find the gene_ids you want: scores above 262: 350124,181648,99462,359615,175809,53297
* then do anvi-get-sequences-from-gene_ids again except this time feed it a list of the gene_ids you want. 

anvi-get-sequences-for-gene-calls -c /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies/drep/dereplicated-genomes_contigs.db -o /groups/banfield/projects/industrial/nelson_lab/arstagnation/assemblies_analysis/amoA_sequences.fa --gene-caller-ids 350124,181648,99462,359615,175809,53297 --delimiter ,

* That will give you the nucleotide sequences you can use to search the primers against in Benchling.


# hmmsearch --cpu 6 --tblout ./scnpilot_dereplicated_all_bacteria_proteins.faa-vs-tigrfam_subset.out --cut_nc ~/scnpilot/hmms/tigrfam_subset.hmm scnpilot_dereplicated_all_bacteria_proteins.faa